## Cruzamento dos Dados (Enunciado Questões + Microdados)

* **Input**: Local do CSV de Questões Extraídas + Local do CSV de Microdados

* **Output**: CSV com as colunas -> *numero_questao*, *enunciado*, *alternativas*, *nu_param_B* e *gabarito*

In [29]:
import pandas as pd

In [30]:
# Definição de Variáveis de Input e Output
year = 2021

question_path = f"../data/extracted_questions/enem_{year}.csv"
microdados_path = f"../data/microdados/microdados_{year}.csv"
output_path = f"../data/merged_data/enem_{year}.csv"

In [31]:
# Ler o CSV extraído do PDF (resultado de Text_Extraction.ipynb)
df_questions = pd.read_csv(question_path, encoding="utf-8")

# Ler o CSV de microdados
df_microdados = pd.read_csv(microdados_path, delimiter=";", encoding="latin1")

In [32]:
# Filtrar os microdados para SG_AREA 'CN', TX_COR 'AZUL' e CO_PROVA 'X'
# ATENÇÃO: O Código da Prova para a Aplicação Regular muda a cada ano
df_microdados_filtrado = df_microdados[
    (df_microdados["SG_AREA"] == "CN")  # Ciência das Naturezas
    & (df_microdados["TX_COR"].str.upper() == "AZUL")  # Apenas caderno azul
    & (df_microdados["CO_PROVA"] == 909)  # Aplicação Regular
]

# Selecionar apenas as colunas de interesse dos microdados
df_microdados_sel = df_microdados_filtrado[
    ["CO_POSICAO", "NU_PARAM_B", "TX_GABARITO"]
].copy()

In [33]:
# Certificar que os tipos das chaves de junção são compatíveis
df_questions["numero_questao"] = df_questions["numero_questao"].astype(str)
df_microdados_sel["CO_POSICAO"] = df_microdados_sel["CO_POSICAO"].astype(str)

# Fazer o cruzamento usando o número da questão (numero_questao e CO_POSICAO)
df_merged = pd.merge(
    df_questions,
    df_microdados_sel,
    left_on="numero_questao",
    right_on="CO_POSICAO",
    how="left",
)

In [34]:
# Remover a coluna CO_POSICAO (redundante)
df_merged.drop(columns=["CO_POSICAO"], inplace=True)

# Renomeando colunas
df_merged.rename(columns={
    "NU_PARAM_B": "nu_param_B",
    "TX_GABARITO": "gabarito"
}, inplace=True)


# Salvando DF resultante
df_merged.to_csv(output_path, index=False, encoding="utf-8")

print("Merge concluído.")

Merge concluído.


In [35]:
df = pd.read_csv(output_path)
df.head()

,numero_questao,enunciado,alternativas,nu_param_B,gabarito
0,92,O sino dos ventos é composto por várias barras...,A: f < f e v < v .; B: f < f e v = v .; ...,1.65693,D
1,93,O polvo mimético apresenta padrões cromáticos ...,A: originaram-se do mesmo ancestral que esses ...,0.64628,E
2,94,A figura foi extraída de um antigo jogo para c...,A: 30 m/s.; B: 35 m/s.; C: 40 m/s.; D: 45 m/s....,3.24395,C
3,95,Os pesticidas organoclorados foram amplamente ...,"A: 3,9 g.; B: 31,2 g.; C: 62,5 g.; D: 125,0 g....",0.76309,A
4,96,A sequência de nucleotídeos do RNA mensageiro ...,A: Deleção de G no códon 3.; B: Substituição d...,1.12828,B
